In [1]:
import os
import tensorflow as tf
from keras import optimizers
from keras.layers import InputLayer
from keras.models import Model

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.chdir("/tf/notebooks/src")

2023-05-22 11:51:58.630799: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from models import SiameseModel
from utils import load_data, visualise

In [3]:
from keras.models import Sequential
from keras.layers import InputLayer, Input
from NetVLAD import NetVLAD

model = Sequential()
model.add(InputLayer(input_shape=(100, 100, 3)))

netvlad_layer = NetVLAD(input_shape=(100, 100, 3))

model.add(netvlad_layer)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 net_vlad (NetVLAD)          (None, 192)               448       
                                                                 
Total params: 448
Trainable params: 448
Non-trainable params: 0
_________________________________________________________________


2023-05-22 11:51:59.937851: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2023-05-22 11:51:59.937878: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: db27e67b67f1
2023-05-22 11:51:59.937883: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: db27e67b67f1
2023-05-22 11:51:59.937943: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: NOT_FOUND: was unable to find libcuda.so DSO loaded into this program
2023-05-22 11:51:59.937962: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 525.105.17


In [8]:
model = SiameseModel()

TypeError: Error converting shape to a TensorShape: Dimension value must be integer or None or have an __index__ method, got value '(224, 224, 3)' with type '<class 'tuple'>'.

In [17]:
from NetVLAD import NetVLAD

input = InputLayer(input_shape=(100, 100, 3))
netvlad = NetVLAD(input_shape=input.output_shape[1:])(input)

model = Model(input=input, output=netvlad)


Building  None


AttributeError: The layer "net_vlad_5" has never been called and thus has no defined input shape. Note that the `input_shape` property is only available for Functional and Sequential models.

In [ ]:
print("Tensorflow version:", tf.__version__)
gpus = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpus))

In [ ]:
BATCH_SIZE = 16
train_data = load_data(anchor_images_path="/tf/CVUSA/terrestrial",
                       positive_images_path="/tf/CVUSA/satellite",
                       batch_size=BATCH_SIZE)
visualise(*list(train_data.take(1).as_numpy_iterator())[0])

In [ ]:
# Checkpoint during training
model = SiameseModel()
model.compile(optimizer=optimizers.Adam(0.001),
              weighted_metrics=[])

In [ ]:
model.fit(train_data,
          epochs=1)

In [ ]:
tf.saved_model.save(model.siamese_network, "/tf/notebooks/resnet")

In [ ]:
model.siamese_network.save("/tf/notebooks/resnet_2")

In [ ]:
cvmNet = tf.keras.models.load_model("/tf/notebooks/src/trained_models/CVM-Net_model/CVM-NET-I/CVM-Net-I_model")